In [2]:
"""
    Convert excel to markdown
"""
import os
from pathlib import Path
from docling.document_converter import DocumentConverter

file_path = '../data/Banking Statement.xlsx'

def get_docling_result(path):
    if not os.path.exists(path):
        print(f"Error: File does not exist {path}")
        return None

    try:
        converter = DocumentConverter()
        markdown_text = converter.convert(path)
        return markdown_text
    except Exception as e:
        print(f"Error while converting: {e}")
        return None

markdown_data = get_docling_result(file_path)
if markdown_data:
    result = markdown_data.document.export_to_markdown()
    print(f"Convert to markdown successfully")
    # print(result)
else:
    print(f"Parse markdown was getting error")


Convert to markdown successfully


In [4]:
"""
    Extract 'Thông tin đối soát & Chi tiết giao dịch' in markdown
"""
import re

def format_currency(value):
    formatted = "{:,.2f}".format(value)
    return formatted.replace(',', '.')

def parse_money_from_text(text):
    if not text or str(text).lower() == 'none': return 0.00
    clean_val = str(text).replace(',', '').strip()
    match = re.search(r"(\d+\.?\d*)", clean_val)
    if match:
        return float(match.group(1))
    return 0.00

def extract_banking_data(doc_result):
    if not doc_result or not doc_result.document:
        return ""

    md_text = doc_result.document.export_to_markdown()

    def find_meta(label):
        match = re.search(rf"{label}\s*:\s*\|?\s*([^|\n]+)", md_text)
        return match.group(1).strip() if match else "N/A"

    common_meta = {
        "bank_owner": find_meta("Chủ tài khoản"),
        "account_num": find_meta("Số tài khoản"),
        "cif": find_meta("CIF"),
        "address": find_meta("Địa chỉ").replace(',', '').replace('\n', ' ').strip(),
        "currency": "VND"
    }

    output_strings = []

    for table in doc_result.document.tables:
        df = table.export_to_dataframe(doc=doc_result.document)

        for _, row in df.iterrows():
            cells = [str(c).strip() for c in row.values]

            if any(re.search(r"\d{4}-\d{2}-\d{2}", cell) for cell in cells[:1]):
                try:
                    date_raw = cells[0].split(' ')[0]
                    d_p = date_raw.split('-')
                    date_str = f"{d_p[2]}/{d_p[1]}/{d_p[0]}"

                    ref_num = cells[1]
                    net_val = parse_money_from_text(cells[3])
                    description = cells[4] if len(cells) > 4 else ""

                    tax_match = re.search(r"VAT Amt:.*?=\s*([\d,.]+)\s*VND", description)
                    tax_val = parse_money_from_text(tax_match.group(1)) if tax_match else 0.00

                    gross_match = re.search(r"Gross Amt:.*?=\s*([\d,.]+)\s*VND", description)
                    gross_val = parse_money_from_text(gross_match.group(1)) if gross_match else net_val

                    if gross_val == 0 and net_val > 0:
                        gross_val = net_val

                    record = (
f"Thông tin đối soát:\n"
f"    Ngày giao dịch: {date_str}\n"
f"    Số tham chiếu: {ref_num}\n"
f"    Số tiền ghi có: {format_currency(net_val)}\n"
f"    VAT Amt: {format_currency(tax_val)} VND\n"
f"    Gross Amt: {format_currency(gross_val)} VND\n\n"
f"Chi tiết giao dịch:\n"
f"    Chủ tài khoản: {common_meta['bank_owner']}\n"
f"    Số tài khoản: {common_meta['account_num']}\n"
f"    CIF: {common_meta['cif']}\n"
f"    Địa chỉ: {common_meta['address']}\n"
f"    Loại tiền: {common_meta['currency']}\n"
f"    Mô tả: {description}"
                    )

                    output_strings.append(record)
                except Exception:
                    continue

    return "\n\n---\n\n".join(output_strings)

final_string_data = extract_banking_data(markdown_data)
print(final_string_data)

Thông tin đối soát:
    Ngày giao dịch: 30/04/2024
    Số tham chiếu: 5219 - 65354
    Số tiền ghi có: 156.000.00
    VAT Amt: 0.00 VND
    Gross Amt: 156.000.00 VND

Chi tiết giao dịch:
    Chủ tài khoản: CT TNHH TM& DV SIEU THI BIG C AN LAC
    Số tài khoản: 71004403751
    CIF: 101346
    Địa chỉ: 1231 QLO 1AKP5F.BINHTRI DONG BBINHTAN
    Loại tiền: VND
    Mô tả: TKP#P021071000143#333684.300424.230435.nop tien pmh cty 125tr go ba ria

---

Thông tin đối soát:
    Ngày giao dịch: 30/04/2024
    Số tham chiếu: 9925 - 34743
    Số tiền ghi có: 9.798.337.00
    VAT Amt: 9.907.00 VND
    Gross Amt: 9.907.318.00 VND

Chi tiết giao dịch:
    Chủ tài khoản: CT TNHH TM& DV SIEU THI BIG C AN LAC
    Số tài khoản: 71004403751
    CIF: 101346
    Địa chỉ: 1231 QLO 1AKP5F.BINHTRI DONG BBINHTAN
    Loại tiền: VND
    Mô tả: T/t T/ung the VISA:SIEU THI BIG C AN LAC BA RIA; MerchNo:11500000457 Gross Amt: Not On-Us=9,907,318.00 VND; VAT Amt:9,907,318.00*1.100%/11 = 9,907.00 VND(VAT code:0301472278)

In [9]:
"""
    Encoding 'Thông tin đối soát' to vector 384 dimensions
    Define schema for vector_db & full_metadata = 'Thông tin đối soát + Chi tiết giao dịch'
    Save into Milvus DB
"""

import os
import re
import time
from pymilvus import MilvusClient
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')
def process_and_store_to_milvus(final_string_output):
    blocks = [block.strip() for block in final_string_output.split('---') if block.strip()]
    if not blocks:
        print("No data found.")
        return None

    reconcle_list = []
    full_metadata_list = []
    for block in blocks:
        match = re.search(r"Thông tin đối soát:\s*(.*?)(?=Chi tiết giao dịch:|$)", block, re.DOTALL)
        if match:
            recon_content = match.group(1).strip()
            reconcle_list.append(recon_content)
            full_metadata_list.append(block)

    db_name = "../data/vector_db.db"
    COLLECTION_NAME = "reconciliation"

    client = MilvusClient(db_name)

    if client.has_collection(COLLECTION_NAME):
        print(f"Deleting existing collection: '{COLLECTION_NAME}'...")
        client.drop_collection(COLLECTION_NAME)

    print(f"Creating vector for {len(reconcle_list)} transactions...")
    embeddings = model.encode(reconcle_list)

    for i, vector in enumerate(embeddings):
        formatted_dims = [f"{val:.4f}" for val in vector[:10]]
        print(f"Vector {i} - Dim: {len(embeddings[i])} - Show 10 Dim: {formatted_dims}")
    print("-" * 100 + "\n")

    client.create_collection(
        collection_name=COLLECTION_NAME,
        dimension=384,
        metric_type="COSINE",
        auto_id=True
    )

    insert_data = [
        {
            "vector": embeddings[i],
            "metadata_detail": full_metadata_list[i]
        }
        for i in range(len(reconcle_list))
    ]

    client.insert(collection_name=COLLECTION_NAME, data=insert_data)
    print(f"All vectors created into {db_name} successfully.")

    return client

client = process_and_store_to_milvus(final_string_data)



Deleting existing collection: 'reconciliation'...
Creating vector for 42 transactions...
Vector 0 - Dim: 384 - Show 10 Dim: ['-0.1167', '0.3514', '-0.0406', '-0.0534', '-0.2323', '-0.2397', '0.0092', '0.3175', '0.0811', '-0.0321']
Vector 1 - Dim: 384 - Show 10 Dim: ['-0.1015', '0.2681', '-0.0791', '-0.0608', '-0.2073', '-0.1581', '-0.0077', '0.2905', '0.1174', '-0.0211']
Vector 2 - Dim: 384 - Show 10 Dim: ['-0.1137', '0.2831', '-0.0826', '-0.0309', '-0.1938', '-0.1869', '-0.0157', '0.3066', '0.1054', '-0.0421']
Vector 3 - Dim: 384 - Show 10 Dim: ['-0.0977', '0.2867', '-0.0797', '-0.0414', '-0.2054', '-0.1973', '0.0119', '0.3280', '0.1389', '-0.0344']
Vector 4 - Dim: 384 - Show 10 Dim: ['-0.1070', '0.3169', '-0.0402', '-0.0586', '-0.2017', '-0.2156', '-0.0157', '0.3585', '0.0845', '-0.0213']
Vector 5 - Dim: 384 - Show 10 Dim: ['-0.1323', '0.3559', '-0.0678', '-0.0447', '-0.1851', '-0.1861', '0.0099', '0.3090', '0.0863', '-0.0064']
Vector 6 - Dim: 384 - Show 10 Dim: ['-0.1056', '0.3008',

In [10]:
"""
    Encoding the embedding vector for Query
    Search by Cosine Similarity in Milvus
    Set the threshold for Cosine Score
"""
def search_transaction(query_text, client):
    query_vector = model.encode([query_text.strip()])

    results = client.search(
        collection_name="reconciliation",
        data=query_vector,
        limit=1,
        output_fields=["metadata_detail"]
    )

    if results and results[0]:
        res = results[0][0]
        score = res['distance']

        print(f"\nCosine: {score:.6f}")

        if score > 0.999:
            print("✅ Result: 100% Matched")
        else:
            print("⚠️ Result: Un-Matched")

        print("-" * 50)
        print(res['entity']['metadata_detail'])

query_transaction = """

    Ngày giao dịch: 30/04/2024
    Số tham chiếu: 9925 - 34743
    Số tiền ghi có: 9.798.337.00
    VAT Amt: 9.907.00 VND
    Gross Amt: 9.907.318.00 VND

"""

search_transaction(query_transaction, client)


Cosine: 1.000000
✅ Result: 100% Matched
--------------------------------------------------
Thông tin đối soát:
    Ngày giao dịch: 30/04/2024
    Số tham chiếu: 9925 - 34743
    Số tiền ghi có: 9.798.337.00
    VAT Amt: 9.907.00 VND
    Gross Amt: 9.907.318.00 VND

Chi tiết giao dịch:
    Chủ tài khoản: CT TNHH TM& DV SIEU THI BIG C AN LAC
    Số tài khoản: 71004403751
    CIF: 101346
    Địa chỉ: 1231 QLO 1AKP5F.BINHTRI DONG BBINHTAN
    Loại tiền: VND
    Mô tả: T/t T/ung the VISA:SIEU THI BIG C AN LAC BA RIA; MerchNo:11500000457 Gross Amt: Not On-Us=9,907,318.00 VND; VAT Amt:9,907,318.00*1.100%/11 = 9,907.00 VND(VAT code:0301472278); Code:1005; SLGD: Not On-Us=13; Ngay 30/04/2024.
